# Replication Draft

## Pull from Markit and construct CDS-related columns

In [1]:
import wrds
WRDS_USERNAME="ygxu"

db = wrds.Connection(wrds_username = WRDS_USERNAME)

## this is the df that I finally used 
query="""
    SELECT ticker, redcode, DATE_TRUNC('month', date) AS month, MIN(date) AS first_trade_date, AVG(parspread) AS midspread
    FROM markit.cds2001
    WHERE tenor = '5Y', country = "United States'
    GROUP BY month, ticker, redcode
    """
#db.raw_sql(query)
cds01_df = db.raw_sql(query)

Loading library list...
Done


In [53]:
cds01_df.isna().sum()

ticker               0
redcode             51
month                0
first_trade_date     0
midspread            0
dtype: int64

In [2]:
cds01_df.head(5)

,ticker,redcode,month,first_trade_date,midspread
0,EMN,29EB75,2001-08-01 00:00:00-04:00,2001-08-10,0.011089
1,CROATI,248DEC,2001-08-01 00:00:00-04:00,2001-08-01,0.016329
2,TIN,8FGHEE,2001-05-01 00:00:00-04:00,2001-05-01,0.011838
3,LNC,5I759F,2001-11-01 00:00:00-05:00,2001-11-01,0.004350
4,CITOH,4ABC88,2001-07-01 00:00:00-04:00,2001-07-02,0.019080


In [3]:
cds01_df["prev_spread"] = cds01_df.groupby("ticker")["midspread"].shift(1)

In [12]:
len(cds01_df.month.unique())

12

## Calculate RD

In [13]:
import pandas as pd 
file_path = "../data_manual/feds200628.csv"
df = pd.read_csv(file_path,skiprows=9)
df["Date"] = pd.to_datetime(df["Date"])

In [14]:
df.head(5)

,Date,BETA0,BETA1,BETA2,BETA3,SVEN1F01,SVEN1F04,SVEN1F09,SVENF01,SVENF02,...,SVENY23,SVENY24,SVENY25,SVENY26,SVENY27,SVENY28,SVENY29,SVENY30,TAU1,TAU2
0,1961-06-14,3.917606,-1.277955,-1.949397,0.0,3.8067,3.9562,NaN,3.5492,3.8825,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.339218,-999.99
1,1961-06-15,3.978498,-1.257404,-2.247617,0.0,3.8694,4.0183,NaN,3.5997,3.9460,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.325775,-999.99
2,1961-06-16,3.984350,-1.429538,-1.885024,0.0,3.8634,4.0242,NaN,3.5957,3.9448,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.348817,-999.99
3,1961-06-19,4.004379,-0.723311,-3.310743,0.0,3.9196,4.0447,NaN,3.6447,3.9842,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.282087,-999.99
4,1961-06-20,3.985789,-0.900432,-2.844809,0.0,3.8732,4.0257,NaN,3.5845,3.9552,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.310316,-999.99


In [3]:
rf_columns = ["SVENY01", "SVENY02", "SVENY03", "SVENY04","SVENY05","SVENY06", "SVENY07","SVENY08","SVENY09", "SVENY10"]
maturities = [i for i in range(0, 11)]

In [4]:
file_path = "../data_manual/DGS3MO.csv"
df3 = pd.read_csv(file_path)

file_path = "../data_manual/DGS6MO.csv"
df6 = pd.read_csv(file_path)

df_treasury = pd.merge(df3, df6, on = "observation_date")
df_treasury.head(5)
df_treasury["Date"] = pd.to_datetime(df_treasury["observation_date"])

In [5]:
df_treasury = df_treasury.drop(columns="observation_date")
df_treasury.head(5)

,DGS3MO,DGS6MO,Date
0,17.01,17.17,1981-09-01
1,16.65,17.32,1981-09-02
2,16.96,17.42,1981-09-03
3,16.64,17.37,1981-09-04
4,NaN,NaN,1981-09-07


In [6]:
rf_columns = ["Date", "SVENY01", "SVENY02", "SVENY03", "SVENY04","SVENY05","SVENY06", "SVENY07","SVENY08","SVENY09", "SVENY10"]
df_rf = df_treasury.merge(df[rf_columns], on = "Date").set_index("Date")
df_rf.head(5)

,DGS3MO,DGS6MO,SVENY01,SVENY02,SVENY03,SVENY04,SVENY05,SVENY06,SVENY07,SVENY08,SVENY09,SVENY10
Date,,,,,,,,,,,,
1981-09-01,17.01,17.17,16.1930,15.7597,15.3419,15.0393,14.8322,14.6891,14.5866,14.5104,14.4516,14.4047
1981-09-02,16.65,17.32,16.2582,15.8046,15.3635,15.0539,14.8447,14.6996,14.5946,14.5156,14.4541,14.4049
1981-09-03,16.96,17.42,16.3433,15.8456,15.3940,15.0987,14.9074,14.7772,14.6838,14.6137,14.5592,14.5156
1981-09-04,16.64,17.37,16.4590,15.8969,15.4284,15.1286,14.9355,14.8042,14.7100,14.6392,14.5842,14.5401
1981-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_rf = df_rf.sort_values(by='Date')

In [8]:
df_rf.columns.to_list()

['DGS3MO',
 'DGS6MO',
 'SVENY01',
 'SVENY02',
 'SVENY03',
 'SVENY04',
 'SVENY05',
 'SVENY06',
 'SVENY07',
 'SVENY08',
 'SVENY09',
 'SVENY10']

In [21]:
def duration_mapper(lst):
    mapper = {"DGS3MO":"0.25", "DGS6MO": "0.5"}
    for col in lst:
        if col.endswith("MO") is False:
            mapper[col] = col[-2:]
    return mapper

In [22]:
mapper = duration_mapper(df_rf.columns.to_list())
mapper

{'DGS3MO': '0.25',
 'DGS6MO': '0.5',
 'SVENY01': '01',
 'SVENY02': '02',
 'SVENY03': '03',
 'SVENY04': '04',
 'SVENY05': '05',
 'SVENY06': '06',
 'SVENY07': '07',
 'SVENY08': '08',
 'SVENY09': '09',
 'SVENY10': '10'}

In [32]:
df_rf = df_rf / 100
df_rf.head()

,DGS3MO,DGS6MO,SVENY01,SVENY02,SVENY03,SVENY04,SVENY05,SVENY06,SVENY07,SVENY08,SVENY09,SVENY10
Date,,,,,,,,,,,,
1981-09-01,0.1701,0.1717,0.161930,0.157597,0.153419,0.150393,0.148322,0.146891,0.145866,0.145104,0.144516,0.144047
1981-09-02,0.1665,0.1732,0.162582,0.158046,0.153635,0.150539,0.148447,0.146996,0.145946,0.145156,0.144541,0.144049
1981-09-03,0.1696,0.1742,0.163433,0.158456,0.153940,0.150987,0.149074,0.147772,0.146838,0.146137,0.145592,0.145156
1981-09-04,0.1664,0.1737,0.164590,0.158969,0.154284,0.151286,0.149355,0.148042,0.147100,0.146392,0.145842,0.145401
1981-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
x = [0.25,0.5,1,2,3,4,5,6,7,8,9,10]

In [40]:
df_rf.loc["2001-01-02"]

DGS3MO     0.058700
DGS6MO     0.055800
SVENY01    0.052389
SVENY02    0.049120
SVENY03    0.047983
SVENY04    0.047879
SVENY05    0.048272
SVENY06    0.048895
SVENY07    0.049608
SVENY08    0.050337
SVENY09    0.051042
SVENY10    0.051704
Name: 2001-01-02 00:00:00, dtype: float64

In [46]:
y = df_rf.loc["2001-05-01"].values

In [47]:
y

array([0.0392    , 0.0392    , 0.04016926, 0.04276494, 0.04522787,
       0.04737815, 0.04926518, 0.05092615, 0.05238759, 0.05367055,
       0.05479288, 0.05577023])

In [48]:
cds01_df.head(5)

,ticker,redcode,month,first_trade_date,midspread
0,EMN,29EB75,2001-08-01 00:00:00-04:00,2001-08-10,0.011089
1,CROATI,248DEC,2001-08-01 00:00:00-04:00,2001-08-01,0.016329
2,TIN,8FGHEE,2001-05-01 00:00:00-04:00,2001-05-01,0.011838
3,LNC,5I759F,2001-11-01 00:00:00-05:00,2001-11-01,0.004350
4,CITOH,4ABC88,2001-07-01 00:00:00-04:00,2001-07-02,0.019080


In [49]:
import numpy as np
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt

f = CubicSpline(x, y, bc_type="natural")
xvals = np.linspace(0,10,41)
yinterp = f(xvals)
b = {'Duration': xvals, 'Yield': yinterp}
data = pd.DataFrame(data=b)
data

,Duration,Yield
0,0.00,0.039200
1,0.25,0.039200
2,0.50,0.039200
3,0.75,0.039563
4,1.00,0.040169
5,1.25,0.040819
6,1.50,0.041471
7,1.75,0.042120
8,2.00,0.042765
9,2.25,0.043402


ZeroDivisionError: integer modulo by zero

## Calculate on daily basis 

In [50]:
import numpy as np
df_rf= df_rf.replace([np.inf, -np.inf], np.nan) 
df_rf= df_rf.fillna(method="ffill").fillna(method="bfill")

C:\Users\Yangge Xu\AppData\Local\Temp\ipykernel_19812\349581229.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_rf= df_rf.fillna(method="ffill").fillna(method="bfill")


,DGS3MO,DGS6MO,SVENY01,SVENY02,SVENY03,SVENY04,SVENY05,SVENY06,SVENY07,SVENY08,SVENY09,SVENY10
Date,,,,,,,,,,,,
1981-09-01,17.01,17.17,16.1930,15.7597,15.3419,15.0393,14.8322,14.6891,14.5866,14.5104,14.4516,14.4047
1981-09-02,16.65,17.32,16.2582,15.8046,15.3635,15.0539,14.8447,14.6996,14.5946,14.5156,14.4541,14.4049
1981-09-03,16.96,17.42,16.3433,15.8456,15.3940,15.0987,14.9074,14.7772,14.6838,14.6137,14.5592,14.5156
1981-09-04,16.64,17.37,16.4590,15.8969,15.4284,15.1286,14.9355,14.8042,14.7100,14.6392,14.5842,14.5401
1981-09-07,16.64,17.37,16.4590,15.8969,15.4284,15.1286,14.9355,14.8042,14.7100,14.6392,14.5842,14.5401
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-17,4.34,4.32,4.2389,4.2362,4.2455,4.2644,4.2908,4.3227,4.3585,4.3969,4.4368,4.4770
2025-02-18,4.34,4.34,4.2622,4.2758,4.2977,4.3261,4.3593,4.3960,4.4350,4.4753,4.5159,4.5563
2025-02-19,4.34,4.34,4.2453,4.2504,4.2672,4.2931,4.3260,4.3639,4.4052,4.4484,4.4925,4.5365


In [87]:
quarter_start_dates = pd.date_range(start=df['Date'].min(), end=df['Date'].max(), freq='QS')

In [88]:
days_since_start = (df['Date'] - df['Date'].min()).dt.days
quarter_days_since_start = (quarter_start_dates - df['Date'].min()).days

In [89]:
df_rf.columns[1:].to_list()

['SVENY01',
 'SVENY02',
 'SVENY03',
 'SVENY04',
 'SVENY05',
 'SVENY06',
 'SVENY07',
 'SVENY08',
 'SVENY09',
 'SVENY10',
 'DGS3MO',
 'DGS6MO']

In [90]:
df_rf.head()

,Date,SVENY01,SVENY02,SVENY03,SVENY04,SVENY05,SVENY06,SVENY07,SVENY08,SVENY09,SVENY10,DGS3MO,DGS6MO
0,1981-09-01,16.1930,15.7597,15.3419,15.0393,14.8322,14.6891,14.5866,14.5104,14.4516,14.4047,17.01,17.17
1,1981-09-02,16.2582,15.8046,15.3635,15.0539,14.8447,14.6996,14.5946,14.5156,14.4541,14.4049,16.65,17.32
2,1981-09-03,16.3433,15.8456,15.3940,15.0987,14.9074,14.7772,14.6838,14.6137,14.5592,14.5156,16.96,17.42
3,1981-09-04,16.4590,15.8969,15.4284,15.1286,14.9355,14.8042,14.7100,14.6392,14.5842,14.5401,16.64,17.37
4,1981-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
import numpy as np
rate_columns = df_rf.columns[1:].to_list()
df_rf[rate_columns] = df_rf[rate_columns].replace([np.inf, -np.inf], np.nan) 
df_rf[rate_columns] = df_rf[rate_columns].fillna(method="ffill").fillna(method="bfill")


C:\Users\Yangge Xu\AppData\Local\Temp\ipykernel_26460\860331299.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_rf[rate_columns] = df_rf[rate_columns].fillna(method="ffill").fillna(method="bfill")


In [94]:
import numpy as np
import pandas as pd
from scipy.interpolate import CubicSpline

def convert_maturity_labels(columns):
    """
    Converts maturity labels into numerical values in months.
    Example: 'SVENY05' → 60 months, 'DGS3MO' → 3 months.
    """
    maturity_map = {}
    for col in columns:
        if col.startswith("SVENY"):  # Convert 'SVENYXX' → months
            years = int(col[-2:])  # Extract last 2 digits
            maturity_map[col] = years * 12
        elif col == "DGS3MO":
            maturity_map[col] = 3
        elif col == "DGS6MO":
            maturity_map[col] = 6
        else:
            raise ValueError(f"Unknown maturity label: {col}")
    
    # Sort maturities by increasing order
    sorted_maturities = sorted(maturity_map.items(), key=lambda x: x[1])
    
    # Return sorted column names and maturity values
    sorted_columns, sorted_maturity_values = zip(*sorted_maturities)
    
    return list(sorted_columns), np.array(sorted_maturity_values)


def extrapolate_rates_decaying(rates, decay_type='exp', alpha=0.05):
    """
    Interpolates and extrapolates risk-free rates to quarterly maturities 
    while applying a decaying weight based on maturity.

    Parameters:
    - rates: DataFrame containing rate maturities as columns.
    - decay_type: 'exp' for exponential decay, 'inv' for inverse maturity weighting.
    - alpha: Decay rate for exponential weighting.

    Returns:
    - DataFrame with quarterly-maturity rates for each time index.
    """
    extrapolated_rates_df = pd.DataFrame(
        columns=np.arange(3, 121, 3), index=rates.index
    )

    # Convert column labels to numerical maturities in months (sorted)
    sorted_columns, maturities = convert_maturity_labels(rates.columns)

    # Ensure rates are reordered based on sorted maturities
    rates = rates[sorted_columns]

    # Define weight function based on decay type
    if decay_type == 'exp':
        weights = np.exp(-alpha * maturities)  # Exponential decay
    elif decay_type == 'inv':
        weights = 1 / maturities  # Inverse maturity weighting
    else:
        raise ValueError("Invalid decay_type. Choose 'exp' or 'inv'.")

    # Normalize weights so they sum to 1
    weights /= weights.sum()

    for index, row in rates.iterrows():
        rates_values = row.values * weights  # Apply decaying weights

        # Cubic spline interpolation (maturities are now strictly increasing)
        cs = CubicSpline(maturities, rates_values, bc_type='natural')

        # Generate quarterly maturities (3, 6, 9, ..., 120 months)
        quarterly_maturities = np.arange(3, 121, 3)
        extrapolated_rates = cs(quarterly_maturities)

        extrapolated_rates_df.loc[index] = extrapolated_rates

    return extrapolated_rates_df

# Example Usage
decay_type = 'exp'  # Choose 'exp' for exponential or 'inv' for inverse maturity
alpha = 0.05  # Controls decay speed in exponential weighting

extrapolated_rates = extrapolate_rates_decaying(df_rf.iloc[:,1:], decay_type=decay_type, alpha=alpha)


# Example Usage
decay_type = 'exp'  # Choose 'exp' for exponential or 'inv' for inverse maturity
alpha = 0.05  # Controls decay speed in exponential weighting

extrapolated_rates = extrapolate_rates_decaying(df_rf.iloc[:,1:], decay_type=decay_type, alpha=alpha)


In [99]:
import numpy as np
import pandas as pd

# Step 1: Extract interpolated quarterly risk-free rates
df_interpolated = extrapolated_rates  # Already computed interpolated rates

# Step 2: Compute λ (hazard rate) from CDS spreads
CDS = 0.01  # Example CDS spread (1%)
L = 0.6  # Loss given default (60%)
lambda_value = 4 * np.log(1 + CDS / (4 * L))  # Compute λ from CDS spreads

# Step 3: Compute RD_t for each quarter separately
rd_per_quarter = []

for t in range(1, len(df_interpolated) + 1):  # Iterate over each quarter
    quarterly_rates = df_interpolated.iloc[:t].mean(axis=1).values  # Use rates up to quarter t
    
    RD_t = (1 / 4) * sum(
        np.exp(-j * lambda_value / 4) * np.exp(-j * quarterly_rates[j-1] / 4)
        for j in range(1, t + 1)
    )
    
    rd_per_quarter.append(RD_t)

# Convert to DataFrame
df_RD = pd.DataFrame({"Date": df_interpolated.index, "RD_t": rd_per_quarter})
print(df_RD.head())  # View first few results


   Date      RD_t
0     0  0.199798
1     1  0.359493
2     2  0.486591
3     3  0.588033
4     4  0.668996


In [100]:
len(df_RD["RD_t"])

11344